# Example application
The example (code and data) shown here are available in the 
`/docs/example` directory of the [HydroBM repository](https://github.com/wknoben/hydrobm).


## Example data
This example uses data obtained from the Caravan data set (Kratzert et al., 2023). To reduce repository size only the "total_precipitation_sum", "temperature_2m_mean", and "streamflow" variables are retained in `camels_01022500_minimal.nc`.

### Subsetting code
For full reproducibility, download the original Caravan data and use this code to subset.

```
import numpy as np
import xarray as xr

# Load the data
data_file = 'camels_01022500.nc'
mini_file = 'camels_01022500_minimal.nc'

data = xr.open_dataset(data_file)
keep = ['date', 'streamflow', 'total_precipitation_sum', 'temperature_2m_mean']
mini = data.drop_vars([var for var in data.variables if not var in keep])
mask = np.isnan(mini['streamflow'])
mini = mini.isel(date=~mask)
mini.to_netcdf(mini_file)

```

### References
Kratzert, F., Nearing, G., Addor, N., Erickson, T., Gauch, M., Gilon, O., Gudmundsson, L., Hassidim, A., Klotz, D., Nevo, S., Shalev, G., and Matias, Y.: Caravan - A global community dataset for large-sample hydrology, Scientific Data, 10, 61, https://doi.org/10.1038/s41597-023-01975-w, 2023.

## Example application

In [ ]:
from hydrobm.calculate import calc_bm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# Get the example data
data_file = './camels_01022500_minimal.nc'
data = xr.open_dataset(data_file)

### Create an exploratory plot
This shows that snow likely plays a role in this basin. 

Note that `matplotlib` is not a dependency of `hydrobm` and thus will not be automatically installed if it isn't already present on your system.

In [ ]:
fig,ax = plt.subplots(1,1)
data['total_precipitation_sum'].plot(ax=ax, label='precipitation')
data['streamflow'].plot(ax=ax, label='streamflow')
data['temperature_2m_mean'].plot(ax=ax, label='temperature')
plt.legend();

### Run HydroBM
For this example, we'll calculate all benchmarks and all metrics, as well as estimate a snow melt flux from the precipitation and temperature data. 

First, we'll specify time masks for benchmark calculation and validation. These are arbitrary choices and will depend on your data availability and study purpose.

In [ ]:
# Specify the calculation and evaluation periods, as boolean masks
cal_mask = data['date'].values < np.datetime64('1999-01-01')
val_mask = ~cal_mask

Next, we'll specify the benchmarks we want to calculate. 
We'll go for the full ensemble available in HydroBM at the time of its initial release.

This will trigger a warning when we run HydroBM, because the `annual_mean_flow` and `annual_median_flows` shouldn't be used in combination with a `val_mask`, but we'll accept that for this example. As a consequence, we'll also see some NumPy warnings when HydroBM runs it's benchmark validation for these two benchmarks.

In [ ]:
# Specify the benchmarks and metrics to calculate
benchmarks = [
        # Streamflow benchmarks
        "mean_flow",
        "median_flow",
        "annual_mean_flow",
        "annual_median_flow",
        "monthly_mean_flow",
        "monthly_median_flow",
        "daily_mean_flow",
        "daily_median_flow",

        # Long-term rainfall-runoff ratio benchmarks
        "rainfall_runoff_ratio_to_all",
        "rainfall_runoff_ratio_to_annual",
        "rainfall_runoff_ratio_to_monthly",
        "rainfall_runoff_ratio_to_daily",
        "rainfall_runoff_ratio_to_timestep",

        # Short-term rainfall-runoff ratio benchmarks
        "monthly_rainfall_runoff_ratio_to_monthly",
        "monthly_rainfall_runoff_ratio_to_daily",
        "monthly_rainfall_runoff_ratio_to_timestep",

        # Schaefli & Gupta (2007) benchmarks
        "scaled_precipitation_benchmark",  # equivalent to "rainfall_runoff_ratio_to_daily"
        "adjusted_precipitation_benchmark",
        "adjusted_smoothed_precipitation_benchmark",
    ]

We'll also calculate values for the four statistics available in HydroBM at its initial release. 

Note that automatically calculating certain metric scores is included in HydroBM as a quality-of-life feature, but HydroBM's usage is not limited to these metrics only. HydroBMs main purpose is to create the benchmark simulation time series and return these to the user. If your metric of choice is not (yet) available within HydroBM, simply use HydroBM to generate the benchmark model timeseries and calculate the metric yourself from those.

Of course, you could always consider contributing your metric code to the HydroBM package through its [GitHub reposity](https://github.com/wknoben/hydrobm) :)

In [ ]:
metrics = ['nse', 'kge', 'mse', 'rmse']

Now we're ready to run the whole thing. Because we only have precipitation data, but know that snow is important, we'll also run HydroBM's simple snow model (`calc_snowmelt = True`). When running everything through `calc_bm`, enabling the snow routine will automatically internally use the resulting snow-plus-melt flux to calculate any requested benchmarks.

Note that if you have an external snow model you like better, you can simply run that and feed that timeseries into HydroBM by changing the `precipitation` variable name to that of your own rain-plus-melt timeseries in `data`.

We'll stick with the default values for `snowmelt_threshold` (threshold between rain and snow), `snowmelt_rate` (how quickly does accumulated snow melt) and `optimization_method` (how do we find optimal lag/smoothing values for the APB and ASPB benchmarks), but I've included them here anyway to show how/that they can be adjusted. See the `Usage` part of the documentation for further details.

This is the part where all the work is done (run snow model, create benchmark timeseries, calculate metric values) and running this will take a minute or two. Note that we'll get some warnings from both HydroBM and NumPy because we're combining `val_mask` with two benchmarks that by definition are incompatible with the concept of unseen data.

In [ ]:
# Calculate the benchmarks and scores
benchmarks,scores = calc_bm(
                data,
                # Time period selection
                cal_mask,
                val_mask=val_mask,
                # Variable names in 'data'
                precipitation="total_precipitation_sum",
                streamflow="streamflow",
                # Benchmark choices
                benchmarks=benchmarks,
                metrics=metrics,
                optimization_method="brute_force",
                # Snow model inputs
                calc_snowmelt=True,
                temperature="temperature_2m_mean",
                snowmelt_threshold=0.0,
                snowmelt_rate=3.0,
            )

### Analyze the results
Here's some basic analysis of the resulting benchmarks and their scores. Feel free to build your own analysis from this!

In [ ]:
# Print the scores with some basic formatting applied
for key,val in scores.items():
    if key == 'benchmarks':
        print(f'{key}: {val}')
    else:
        pm = [f'{num:.2f}' for num in val]
        print(f'{key}: {pm}')

Not very clear. Let's create some hydrographs as well.

In [ ]:
# Select the four best benchmarks for plotting
def top_n_indices_and_values(values_list, n=4):
    arr = np.array(values_list) # numpy array
    nan_idx = np.where(np.isnan(arr)) # find nan values
    arr_sort = arr.argsort() # sort the full array, nans go at the end
    arr_sort = arr_sort[~np.in1d(arr_sort, nan_idx)] # remove nans
    indices = arr_sort[-n:] # get the top n indices
    values = arr[indices] # get the values
    return indices.tolist(), values.tolist()

In [ ]:
idx,vals = top_n_indices_and_values(scores['kge_val'], 4)
top_benchmarks = [scores['benchmarks'][id] for id in idx]
top_kge_vals = [scores['kge_val'][id] for id in idx]

In [ ]:
# Print streamflow along with the four best benchmarks
fig,ax = plt.subplots(4,1, figsize=(14,14))
for i,(bm,kge) in enumerate(zip(top_benchmarks,top_kge_vals)):
    data['streamflow'].plot(ax=ax[i], linewidth=2, label='streamflow')
    benchmarks[f'bm_{bm}'].plot(ax=ax[i], label=bm)
    ax[i].legend(loc='upper left')
    ax[i].set_title(f'{bm} (KGE: {kge:.2f})')
    ax[i].set_xlabel('') # drops 'Date'
    
plt.tight_layout()

In [ ]:
# Reformat the scores a bit for cleaner saving 
col_names = scores.pop("benchmarks", None)
df = pd.DataFrame(scores, index=col_names)
df = df.T

In [ ]:
# Uncomment these to save the benchmark models and scores
#  No point doing that here while building the documentation.
#df.to_csv("benchmark_scores.csv")
#benchmarks.to_csv("benchmark_flows.csv")